# Adder
Quantum computers have the function of being able to perform the same calculations as conventional computers. Check the circuit for adder.

## What we'll learn this time
1. Implementing Binary Adder Using Quantum Gates
2. Using quantum superposition, we can realize multiple adder in a single circuit

## Install Blueqat
Install Blueqat from pip.

In [1]:
!pip install blueqat

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


## Addition of binary numbers
Adder uses a CCX gate to perform digit up and a CX gate to perform addition. In this case, we will do a quantum circuit of binary addition, a+b = cd. This time, we will implement four types of addition according to the values of a and b. Each addition is,

0+0 = 00 => 0000  
0+1 = 01 => 0101  
1+0 = 01 => 1001  
1+1 = 10 => 1110  

The first two qubits are the input values a and b, and the second two qubits are the output values c and d. I've built a separate circuit for the input of a and b and a circuit for the addition, and used it several times. For data input, use the X gate to flip 0 to 1.

The circuit part of the additon looks like this. * is a controlled bit.

```
a ---*---*------- a
b ---*---|---*--- b
0 ---X---|---|--- c
0 -------X---X--- d
```

In [2]:
#import tools
from blueqat import Circuit

#additon part
adder = Circuit().ccx[0,1,2].cx[0,3].cx[1,3]

In [3]:
#0+0
(Circuit() + adder).m[:].run(shots=100)

Counter({'0000': 100})

In [4]:
#0+1
(Circuit().x[1] + adder).m[:].run(shots=100)

Counter({'0101': 100})

In [5]:
#1+0
(Circuit().x[0] + adder).m[:].run(shots=100)

Counter({'1001': 100})

In [6]:
#1+1
(Circuit().x[0,1] + adder).m[:].run(shots=100)

Counter({'1110': 100})

As you can see, we were able to add up.

## Addition using superposition
Now, instead of filling in the data one by one with the X-gate, let's use the H-gate to do the addition.

In [7]:
#Use H gate as input instead of X gate
(Circuit().h[0,1] + adder).m[:].run(shots=100)

Counter({'1110': 27, '0101': 18, '0000': 34, '1001': 21})

Using the Hadamal Gate, I got 4 answers, about 1/4 each. If you create a universal addition circuit like this, you can perform calculations using the superposition state.

## Addition using entanglement
Next, instead of the H-gate, we try to add a+b=1 using quantum entanglement.

In [8]:
#Make an entangled state of 01 and 10
(Circuit().h[0].cx[0,1].x[0] + adder).m[:].run(shots=100)

Counter({'0101': 50, '1001': 50})

Since the input values 01 and 10 are entangled, the addition of these two will come out about half by half.

--------

## Advanced: General addition
It implements the addition of common decimal numbers to each other.  
Let's consider the sum of $a, b$.
$a, b$ is $a = a_n ... a_0$, $b = b_n ... b_0$ and can be expressed in binary numbers.  
(where n is based on the larger number.)

The circuit is as following.

<img src="https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F218694%2F489c8ea6-130c-d44b-4bae-4d86f64556c2.png?ixlib=rb-1.2.2&auto=format&gif-q=60&q=75&s=e67fbf71779fa7ebb443e93663c56dc4">

$c_i$ is a carry bit.  
This circuit is divided into two parts: the carry part and the sum part.

we consider these two parts first.

### Carry up

<img src="../tutorial-ja/img/100_0.png" width="30%">

Let $c_i, a_i, b_i, c_{i+1}$ from the top, and $c_{i+1}$ shows the carry-up part.

### Sum

<img src="../tutorial-ja/img/100_1.png" width="30%">

Let $c_i, a_i, b_i$ from the top and the sum of the three numbers will appear in $b_i$.

### implementation

General addition is:

1. Using the carry circuit calculates the carry bit.  
2. Using CX gate undo the last part of the carry.  
3. Using the sum circuit outputs the sum of each position to $b_n$.  
4. Using the reverse carry circuit undo the value of that level.  
5. Using the sum circuit outputs the sum of each position to $b_i$.  
6. repetition of 4,5.

$a+b$ is output in $b_{n+1} ... b_0$.  
First of all, we make the carry and its reverse circuit and the sum circuit.

In [1]:
from blueqat import Circuit

def carry(i):
    return Circuit().ccx[i+1,i+2,i+3].cx[i+1,i+2].ccx[i,i+2,i+3]

def carry_reverse(i):
    return Circuit().ccx[i,i+2,i+3].cx[i+1,i+2].ccx[i+1,i+2,i+3]

def sum(i):
    return Circuit().cx[i+1,i+2].cx[i,i+2]

We'll also create a function to turn a decimal number into a binary number.

In [2]:
def tobinary(A):
    return bin(A)[2:]

tobinary(10)

'1010'

Next, create a function that maps a binary number to a circuit.

In [3]:
def digits(a,b): 
     # change to the binary
     aa = tobinary(a)  
     bb = tobinary(b)  
     alen = len(aa)  
     blen = len(bb)  

     # get the value n
     maxlen = max(alen,blen) 
     if alen>blen: 
         bb = bb.zfill(alen) 
     elif blen>alen: 
         aa = aa.zfill(blen) 

     # mapping
     str = '' 
     for i in range(maxlen): 
         str += '0' + aa[maxlen-i-1] + bb[maxlen-i-1] 
     str += '0' 
     return str

digits(2,2)

'0000110'

The X gate needs to be applied to match the mapped values, because the initial state of the circuit is all zero.

In [4]:
def toX(a): 
     cir = Circuit(len(a)) 
     for i in range(len(a)): 
         if a[i] == "1": 
             cir += Circuit().x[i] 
     return cir

toX("101").m[:].run(shots=100)

Counter({'101': 100})

Finally, consider the output part.  
The output is a binary number, so it is converted to a decimal number.

In [5]:
def todecimal(A):
    return int(str(A),2) 

todecimal(1001)

9

The circuit outputs a mixture of $a_i, b_i$ and $c_i$.  
We get only $b_i$.

In [6]:
def getb(result): 
     str = result[-1] 
     digi = int((len(result)-1)/3) 
     for i in range(digi): 
         str += result[-2-i*3] 
     return todecimal(str)

getb("0000110")

2

### General circuit

In [7]:
def plus(a,b): 
     # Mapping a binary number to a circuit
     qubits = len(digits(a,b)) 
     cir1 = toX(digits(a,b)) 
     digi = int((len(digits(a,b))-1)/3) 

     # first carry circuit
     cir2 = Circuit(qubits)     
     for i in range(digi): 
         cir2 += carry(i*3) 

     # last digit
     cir3 = Circuit(qubits).cx[-3,-2] + sum((digi-1)*3) 

     # Output the sum to bi
     cir4 = Circuit(qubits) 
     for i in range(digi-1): 
         cir4 += carry_reverse((digi-i-2)*3) 
         cir4 += sum((digi-i-2)*3)

     result = (cir1 + cir2 + cir3 + cir4).m[:].run(shots=1) 
     return getb(result.most_common()[0][0])

Let's actually calculate it.

In [8]:
plus(2,2)

4

In [9]:
plus(13,15)

28

In [10]:
plus(70,90)

160

The last calculation takes some time, but we were able to implement a general adder.

### References
V. Vedral, A. Barenco, A. Ekert, "Quantum Networks for Elementary Arithmetic Operations",  
(Submitted on 16 Nov 1995)  
https://arxiv.org/pdf/quant-ph/9511018.pdf